<a href="https://colab.research.google.com/github/Deep-Learning-Qatar/EEG-Vision/blob/master/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pipeline**: This is a general set-up to evaluate our entire EEG-Vision predictive system using the thre individual models that are imported to maximize modularity.

In [ ]:
# mount gdrive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Imports

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

import matplotlib.pyplot as plt
import random

In [ ]:
# install and import wandb
!pip install wandb --upgrade
import wandb
wandb.login()

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.29)


wandb: Currently logged in as: sbaumann (use `wandb login --relogin` to force relogin)


True

In [ ]:
%cd /gdrive/MyDrive/11-785 Deep Learning/Project

# path to data
EEG_dataset_path = '/gdrive/MyDrive/11-785 Deep Learning/Project/EEG_datasets.pth'
Image_dataset_path = '/gdrive/MyDrive/11-785 Deep Learning/Project/data_by_image.pth'

from eeg_net import make_EEG_Net, train_and_val_EEG_Net, test_EEG_Net
from image_net import make_Image_Net, train_and_val_Image_Net, test_Image_Net, make_EEG_to_Image_data_loaders
from eeg_image_map import make_EEG_Image_Map, train_and_val_EEG_Image_Map, test_EEG_Image_Map, calc_feature_id_acc

/gdrive/MyDrive/11-785 Deep Learning/Project


# Pipeline (EEG-Net and Image-Net)

In [ ]:
def model_pipeline_EEG_Net(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='EEG-Net', entity='eegvision', config=hyperparameters):
      config = wandb.config
      model, EEG_data_loaders, criterion, optimizer, scheduler = make_EEG_Net(config, EEG_dataset_path)

      %cd EEG-Net
      model, val_acc_hist = train_and_val_EEG_Net(wandb, config, model, EEG_data_loaders, criterion, optimizer, scheduler, num_epochs=config.num_epochs)

    return model, EEG_data_loaders, val_acc_hist

In [ ]:
EEG_hyperparams = dict(
    model_nr = 2,

    # Models to choose from [fares]
    model = "fares",

    # Number of classes in the dataset
    num_classes = 40,

    # Number of epochs to train for
    num_epochs = 1,

    lr = 0.005,
    weight_decay = 5e-6,
    dropout = 0.3
)

EEG_Net, EEG_data_loaders, eeg_acc_hist = model_pipeline_EEG_Net(EEG_hyperparams)

/gdrive/MyDrive/11-785 Deep Learning/Project


/gdrive/MyDrive/11-785 Deep Learning/Project/EEG-Net
Epoch 1/1
----------
[train] Loss: 3.6624 Acc: 0.0371
[val] Loss: 3.5527 Acc: 0.0377

Training complete in 0m 9s
Best val Acc: 0.037688


epoch,0
train_loss,3.66243
train_acc,0.03706
_runtime,59
_timestamp,1620217124
_step,1
val_loss,3.55271
val_acc,0.03769


epoch,▁▁
train_loss,▁
train_acc,▁
_runtime,▁█
_timestamp,▁█
_step,▁█
val_loss,▁
val_acc,▁


In [ ]:
def model_pipeline_Image_Net(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='Image-Net', entity='eegvision', config=hyperparameters):
      config = wandb.config
      model, Image_data_loaders, criterion, optimizer = make_Image_Net(config, EEG_dataset_path, Image_dataset_path)

      %cd Image-Net
      model, val_acc_hist = train_and_val_Image_Net(wandb, config, model, Image_data_loaders, criterion, optimizer,
                                                    is_inception=(config.model_name=="inception"))

    return model, Image_data_loaders, val_acc_hist

In [ ]:
Image_hyperparams = dict(
    model_nr = 1,

    # Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
    model_name = "inception",

    # Number of classes in the dataset
    num_classes = 40,

    # Batch size for training (change depending on how much memory you have)
    batch_size = 64,

    # Number of epochs to train for
    num_epochs = 1,

    # Flag for feature extracting. When False, we finetune the whole model,
    #   when True we only update the reshaped layer params
    feature_extract = True,

    lr = 0.001,
)

Image_Net, Image_data_loaders, image_acc_hist = model_pipeline_Image_Net(Image_hyperparams)

/gdrive/MyDrive/11-785 Deep Learning/Project


Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias
/gdrive/My Drive/11-785 Deep Learning/Project/Image-Net
Epoch 1/1
----------
[train] Loss: 5.1587 Acc: 0.0682
[val] Loss: 3.4743 Acc: 0.2613

Training complete in 0m 7s
Best val Acc: 0.261307


epoch,0
train_loss,5.15873
train_acc,0.06821
_runtime,105
_timestamp,1620217232
_step,1
val_loss,3.47426
val_acc,0.26131


epoch,▁▁
train_loss,▁
train_acc,▁
_runtime,▁█
_timestamp,▁█
_step,▁█
val_loss,▁
val_acc,▁


# Create feature dataset

In [ ]:
nr_eeg_features, nr_image_features = 128, 40

In [ ]:
def make_feature_datasets(EEG_data_loaders, Image_data_loaders):
    global nr_eeg_features, nr_image_features

    input_size = 299 if Image_hyperparams['model_name'] == 'inception' else 224
    new_Image_data_loaders = make_EEG_to_Image_data_loaders(EEG_data_loaders, Image_dataset_path, input_size)
    
    for phase in ['train_unshuffle', 'val', 'test']:
        # Pass in train, valid, test data through EEG-Net to get EEG features
        _, eeg_features, _ = test_EEG_Net(EEG_Net, EEG_data_loaders[phase])
        eeg_features = [item for sublist in eeg_features for item in sublist]
        nr_eeg_features = len(eeg_features[0])

        # Pass in train, valid, test data through Image-Net to get Image features
        _, image_features, _ = test_Image_Net(Image_Net, new_Image_data_loaders[phase])
        image_features = [item for sublist in image_features for item in sublist]
        nr_image_features = len(image_features[0])

        data_path = phase + '_data.npy'
        np.save(data_path, np.array(eeg_features))

        labels_path = phase + '_labels.npy'
        np.save(labels_path, np.array(image_features))


In [ ]:
%cd /gdrive/MyDrive/11-785 Deep Learning/Project
make_feature_datasets(EEG_data_loaders, Image_data_loaders)

/gdrive/MyDrive/11-785 Deep Learning/Project
nr eeg outputs: 9580
nr eeg outputs: 9580
nr eeg outputs: 1194
nr eeg outputs: 1194
nr eeg outputs: 1191
nr eeg outputs: 1191


# Pipeline (EEG-Image-Map)

In [ ]:
def model_pipeline_EEG_Image_Map(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='EEG-Image-Map', entity='eegvision', config=hyperparameters):
      config = wandb.config
      model, feature_data_loaders, criterion, optimizer, scheduler = make_EEG_Image_Map(config)

      %cd EEG-Image-Map
      model, val_acc_hist = train_and_val_EEG_Image_Map(wandb, config, model, feature_data_loaders, criterion, optimizer, scheduler)

    return model, feature_data_loaders, val_acc_hist

In [2]:
feature_hyperparams = dict(
    model_nr = 1,

    # Models to choose from [mlp, bilstm]
    model = "bilstm",

    train_data_path   = 'train_unshuffle_data.npy',
    train_labels_path = 'train_unshuffle_labels.npy',
    val_data_path     = 'val_data.npy',
    val_labels_path   = 'val_labels.npy',
    test_data_path    = 'test_data.npy',
    test_labels_path  = 'test_labels.npy',
    bidirectional    = True,
    lr               = 2e-2,
    wd               = 5e-6,
    dropout_prob     = 0.1,  
    num_layers       = 4,
    hidden_size      = 300,  
    num_classes      = nr_image_features, #? (same as extractec image features)
    embiddings_num   = nr_eeg_features,  #  (same as number of features extracted from EEG_Net)
    batch_size       = 128,
    patience         = 1,
    scheduler_factor = 0.5,
    num_epochs       = 50,
    mlp_layers       = [128, 64, 40]
)

EEG_Image_Map, feature_data_loaders, val_acc_hist = model_pipeline_EEG_Image_Map(feature_hyperparams)

NameError: ignored

# Evaluate System

In [ ]:
def evaluate_system(verbose=True):
    # Evaluate EEG-Net
    _, _, eeg_acc = test_EEG_Net(EEG_Net, EEG_data_loaders['test'])

    # Evaluate Image-Net
    _, _, image_acc = test_Image_Net(Image_Net, Image_data_loaders['test'])

    # Evaluate EEG-Image-Map
    feature_out, feature_loss = test_EEG_Image_Map(feature_data_loaders['test'])
    feature_out = [item for sublist in feature_out for item in sublist]

    feature_id_acc = calc_feature_id_acc(feature_out, feature_data_loaders['test'])

    if verbose:
        print('EEG-Net test accuracy:', eeg_acc)
        print('Image-Net test accuracy:', image_acc)
        print('EEG-Image-Map test loss:', feature_loss)
        print('EEG-Image-Map identification accuracy:', feature_id_acc)

    return eeg_acc, image_acc, feature_loss, feature_id_acc

In [ ]:
evaluate_system()